In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow keras

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler

# Load the data
data = pd.read_csv('/content/drive/MyDrive/commuter_demand.csv')

data.fillna(0, inplace=True)  # Handle missing values by filling with 0

# Normalize the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['1002', '1003', '1004', '1005', '1006', '1007', '1008']])

# Create input sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        seq = data[i:i+seq_length]
        label = data[i+seq_length, -1]
        X.append(seq)
        y.append(label)
    return np.array(X), np.array(y)

seq_length = 24
X, y = create_sequences(scaled_data, seq_length)

# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create the LSTM model
model = Sequential()
model.add(Input(shape=(seq_length, X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)

Epoch 1/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.0300 - val_loss: 0.0112
Epoch 2/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0104 - val_loss: 0.0071
Epoch 3/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.0080 - val_loss: 0.0068
Epoch 4/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0074 - val_loss: 0.0052
Epoch 5/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0057 - val_loss: 0.0029
Epoch 6/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 7/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 0.0033 - val_loss: 0.0018
Epoch 8/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0023 - val_loss: 0.0024
Epoch 9/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0027 - val_loss: 0.0024
Epoch 10/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - loss: 0.0025 - val_loss: 0.0024
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [5]:
model.save('trained_model.keras')

In [6]:
from google.colab import files
files.download('trained_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>